In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# 確認是否有可用的GPU，盡可能用GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Using device: {device}')


In [ ]:
# 讀取CSV資料，包含 "text" 與 "label" 欄位
df = pd.read_csv("referance_newCat.csv")

# 分類標籤
categories = [
    "人身攻擊與侮辱性言論",
    "質疑身份與背景",
    "存在感與關注度評論",
    "政治立場批評",
    "政治立場表態",
    "政治陰謀論與指控",
    "政績與能力質疑",
    "造謠與誠信質疑",
    "反指標與諷刺預測",
    "罷免相關評論",
    "違建農舍議題",
    "疫情與疫苗相關評論",
    "疑似機器人或複製貼上留言",
    "幽默與嘲諷",
    "簡短情緒表達"
]

# 將文字標籤轉成數字ID
label2id = {cat: idx for idx, cat in enumerate(categories)}
id2label = {idx: cat for cat, idx in label2id.items()}

df['label_id'] = df['label'].apply(lambda x: label2id[x])  # 將文字標籤轉數字


In [ ]:
# 將資料集分為訓練集和測試集，並保持類別比例
# 寫死random_state，讓每次分割都一樣
# 可以確保每次訓練的資料集都是相同的，方便比較模型表現
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label_id'], random_state=42)

print("訓練筆數：", len(train_df))
print("測試筆數：", len(test_df))


In [ ]:
# 這邊其實測試時試了很多不同的模型，像是bert-base-chinese,hfl/chinese-roberta-wwm-ext, hfl/chinese-roberta-wwm-ext-large 等等
# 但最後發現 hfl/chinese-roberta-wwm-ext-large 效果最好，雖然速度慢了一點，但準確率還是有提升
model_name = "hfl/chinese-roberta-wwm-ext-large"  # 使用更大的預訓練模型，chatgpt推薦的，效果看起來也不差(比起原本的bert-base)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 將文字資料轉成 BERT 相容的輸入格式(以上的模型都是吃這個格式)
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="longest",     # 也可以改成 longest，若想動態padding可使用 batched 方式
        truncation=True,
        max_length=512
        )


In [ ]:
# 將資料轉成 Dataset 格式，這樣才能用 Trainer 進行訓練
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df[['text', 'label_id']])
test_dataset  = Dataset.from_pandas(test_df[['text', 'label_id']])

# 進行 Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# 移除多餘欄位，並指定特徵名稱
train_dataset = train_dataset.remove_columns(["text"])
train_dataset = train_dataset.rename_column("label_id", "labels")
train_dataset.set_format("torch")

test_dataset = test_dataset.remove_columns(["text"])
test_dataset = test_dataset.rename_column("label_id", "labels")
test_dataset.set_format("torch")

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})


In [ ]:
num_labels = 15  # 分類數量
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
# 將模型移動到GPU
model.to(device)


In [ ]:
# 設定訓練參數，根據模型的Readme檔案，這邊的參數是參考他們的設定
# 我的電腦batch size只能到3 (3050 laptop 30W)，可以根據自己的電腦調整
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",       # 每個 epoch 結束後做評估
    save_strategy="epoch",            # 每個 epoch 儲存一次模型 (可根據需求調整)
    num_train_epochs=5,               # 以資料量少為前提，先試試 5 epoch
    learning_rate=2e-5,
    per_device_train_batch_size=3,    # 視硬體調整
    per_device_eval_batch_size=3,
    logging_dir="./logs",             # 訓練過程紀錄
    logging_steps=10,
    load_best_model_at_end=True,       # 在最後載入最好的模型權重
    report_to="none"                  
)



In [ ]:
#評估函數，評估每次epoch的準確率
# 這邊使用了evaluate這個套件，這是huggingface官方的評估工具，可以算各種指標(by chatgpt)
import evaluate

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    return {"accuracy": acc["accuracy"]}


In [ ]:
# 自動把短的句子(每筆測資經過tokenizer後，會變成一串串token)用 padding 補齊到一樣的長度，以便放入模型。但是前面tokenizer有改成另外一種形式動態padding，所以理論上不用這段的程式
# 雖然是這樣說但是我這邊也沒有到很理解，該去細細研究一下NLP了(x
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
#將前面所有設定的參數塞進去Trainer裡面
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()

# 取得最佳模型存放的 checkpoint 路徑
best_checkpoint_dir = trainer.state.best_model_checkpoint
print("Best model checkpoint folder:", best_checkpoint_dir)

# 把最佳模型另存到某個資料夾，例如 "./best_model"
best_model_dir = "./best_model"
trainer.save_model(best_model_dir)
print(f"Best model is now saved to: {best_model_dir}")

#到此預訓練結束，接下來是使用模型的部分

In [ ]:
#預訓練完成後，之後想要使用模型的話，只要載入模型就可以了
#理論上若不需要重新訓練的話，可以直接從這邊開始執行(當然前面要import)

best_model_dir = "./best_model"
model_name = "hfl/chinese-roberta-wwm-ext-large"
model = AutoModelForSequenceClassification.from_pretrained(best_model_dir)
tokenizer = AutoTokenizer.from_pretrained(best_model_dir)
model.to(device)

In [ ]:
# 讀取測試資料，格式對應之前爬蟲出來的資料
import pandas as pd

all_list = []

year = '2022'
#input_json_path = f"C://Users//andyw//Desktop//桑鑼的分類//fb_comments//comments//result_{i}.json"
input_csv_path = f"C://Users//andyw//Desktop//AIComments//AIClassify//comments_{year}.csv"

df = pd.read_csv(input_csv_path, header=None, names=["貼文時間","內容", "時間"])

#all_list

In [ ]:

import pandas as pd
import torch
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

all_list = []
years = ['2020', '2019']

for year in years:
    #input_json_path = f"C://Users//andyw//Desktop//桑鑼的分類//fb_comments//comments//result_{i}.json"
    input_csv_path = f"C://Users//andyw//Desktop//AIComments//AIClassify//comments_{year}.csv"

    df = pd.read_csv(input_csv_path, header=None, names=["貼文時間","內容", "時間"])


    # (假設你已經先行載入完模型和 tokenizer)
    # model_name = "你的模型路徑或名稱"
    # tokenizer = AutoTokenizer.from_pretrained(model_name)
    # model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # 假設 test_texts 是一個包含「內容」欄位的 DataFrame
    # e.g. test_texts = pd.read_excel("some_excel_file.xlsx")
    #將內容欄位的空值填上空字串，並轉成字串格式(確保資料一致)
    test_texts = df
    test_texts['內容'] = test_texts['內容'].fillna("").astype(str)
    input_sentences = test_texts['內容'].tolist()


    # 設定批次大小
    batch_size = 16

    all_predictions = []

    # 分批進行預測，並在迴圈外包上 tqdm 以顯示進度(by chatgpt)
    for i in tqdm(range(0, len(input_sentences), batch_size), desc="Predicting"):
        batch_texts = input_sentences[i : i + batch_size]

        # Tokenizer
        encoding = tokenizer(
            batch_texts,
            padding="longest",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            outputs = model(**encoding)
        batch_preds = torch.argmax(outputs.logits, dim=1).cpu().numpy().tolist()

        # 收集每一批的預測結果
        all_predictions.extend(batch_preds)

    # 將數字預測結果轉成文字標籤
    predicted_labels = [id2label[p] for p in all_predictions]

    # 將結果寫回原本 DataFrame
    test_texts['分類'] = predicted_labels

    # (可選) 檢查預測結果
    for text, label in zip(test_texts['內容'][:5], test_texts['分類'][:5]):  # 範例只印前 5 筆
        print(f"文本: {text}\n -> 預測分類: {label}\n")

    # 儲存結果到 Excel 檔
    output_path = f"C://Users//andyw//Desktop//AIComments//AIClassify//resultComment//result{year}.xlsx"
    test_texts.to_excel(output_path, index=False)
    print(f"已將結果儲存至：{output_path}")
